In [ ]:
import numpy as np
import pandas as pd

import strawb
import os
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import scipy.interpolate
import matplotlib.colors
import matplotlib.ticker

# Import Data

In [ ]:
# DataFrame ueber mehrere Tage

cluster_temp_fname = 'TUMPMTSPECTROMETER001.gz'  # database of the camera you want to analyse the pictures of

if not os.path.exists(cluster_temp_fname):
    print("Doesn't exist")
else:
    image_cluster_db = strawb.sync_db_handler.ImageClusterDB(file_name=cluster_temp_fname)

Load files from the ONC server.

In [ ]:
# load DB
db = strawb.SyncDBHandler(file_name='Default')  # loads the db
db.load_onc_db_update(save_db=True)  # update the DB, could take some time if it has to load info. from ONC

In [ ]:
mask = (db.dataframe.deviceCode == 'TUMPMTSPECTROMETER001') # that's the pmtspec module
mask &= (db.dataframe.dataProductCode =='MSSCD') # that's the camera data of the pmtspec module

## select the file for the biolumi event with a window of +- 5 hours
# timestamp = pd.Timestamp('2022-03-04T23:44:09', tz='UTC')  # gain = 30
timestamp = pd.Timestamp('2021-09-04T23:44:09', tz='UTC')  # gain = 50
mask &= db.dataframe.dateFrom >= timestamp - pd.Timedelta('5H')  # - 5 hours
mask &= db.dataframe.dateFrom <= timestamp + pd.Timedelta('5H')  # - 5 hours

### selected one file from the DB (it's the same as the file we selected above by hand)
db.dataframe[mask]

Download the missing files which aren't synced so far from `db.dataframe[mask]`

In [ ]:
if not db.dataframe.synced[mask].all():
    db.update_db_and_load_files(
        db.dataframe[mask],
        output=True,  # print output to console
        download=True,  # download the files
        save_db=True,
    )  # update the DB

Import the file to the Camera Module 

In [ ]:
# select the Camera file(s) -> dataProductCode == 'MSSCD'
item = db.dataframe[mask & (db.dataframe.dataProductCode =='MSSCD')]

try: # if the pmtspec file is still open
    camera.file_handler.close()
except:
    pass
    
# generate a virtual hdf5 to combine the datasets if there are multiple files selected
if len(item) > 1:
    vhdf5 = strawb.VirtualHDF5('MSSCD_event_view.hdf5', item.fullPath.to_list())  
    file_name = vhdf5.file_name
else:
    file_name = item.fullPath[0]

# create an instance of the Camera
camera = strawb.Camera(file_name)

## ----------------------------------------------------------------------------------------------------------

# Adjust Dataframe

In [ ]:
# drop entries with label 0 (background)
df = image_cluster_db.dataframe[image_cluster_db.dataframe.label!=0]  

In [ ]:
# create a mask for the dataframe

limit = 20  # limit from plot
limit_max = 1e5  # limit from plot
parameter = 'n_pixel'

mask_para = df[parameter]  > limit
mask_para &= df[parameter]  < limit_max

In [ ]:
df_cut = df[mask_para]  # keep only entries with 20 < n_pixel < 1e5

##### Import the mask for the mounting of the camera.

In [ ]:
#  get the mounting

from strawb.sensors.camera.config import Config

cam_config = Config(device_code='TUMPMTSPECTROMETER001')
mounting_cam = cam_config.mask_mounting

## ----------------------------------------------------------------------------------------------------------

# Center of Mass/Pixels/Minimum Bounding Box

#### Center of Mass

In [ ]:
# position of the module above the camera for TUMPMTSPECTROMETER001
module_y = (694.507043+693.768910+686.917549+686.774601)/4
module_x = (353.492346+353.288075+357.290158+356.768160)/4

In [ ]:
center_of_mass_y = df_cut['center_of_mass_x'].to_numpy()
center_of_mass_x = df_cut['center_of_mass_y'].to_numpy()

### Scatter plot ###
# def. linear norm function for marker size, maps [c.min(), c.max()] -> [s_min, s_max]
# min and max size of markers
s_min=1.
s_max=5e2
norm_size = scipy.interpolate.interp1d([df_cut.n_pixel.min(), df_cut.n_pixel.max()], [s_min, s_max])

# def. cmap, select one
#cmap=strawb.tools.cmap_manipulator('viridis_r', v_min=.03, v_max=1)  # v_min=.03 - cuts the yellow part
cmap=strawb.tools.cmap_manipulator('viridis', v_min=0, v_max=.97)  # v_max=.97 - cuts the yellow part
#cmap=strawb.tools.cmap_manipulator('Blues', v_min=.35, v_max=1)  #  v_min=.3 - cuts the white part


### PLOT
fig, ax = plt.subplots()
sc = plt.scatter(center_of_mass_x, center_of_mass_y, c=df_cut.n_pixel, s=norm_size(df_cut.n_pixel), 
                 cmap=cmap, norm=matplotlib.colors.LogNorm(),
                 alpha=.3)


# circle around the module above the camera
plt.scatter(module_x, module_y, s=200, facecolors='none', edgecolors='red', 
            label='Module above camera', linewidth=1.5) 
# lines for the data cable and steel cable
ax.plot([625, module_x], [300, module_y], 'r-', linewidth=1.5, label='Data cable')
ax.plot([678, module_x], [320, module_y], 'r', linewidth=1.5, linestyle='--', label = 'Steel cable')
ax.legend(loc='lower right', prop={'size':12})

legend1 = ax.legend(loc='lower left', bbox_to_anchor = (1.35, .27))
ax.add_artist(legend1)


# colorbar
cb = plt.colorbar(sc)
cb.set_label("Size of Cluster [Pixel]")


### LEGEND START
# get the legend - maybe there is a better solution
num = matplotlib.ticker.LogLocator(numticks=6).tick_values(df_cut.n_pixel.min(), df_cut.n_pixel.max())

fmt = "{x:.0f} Pixel"
legends_c = sc.legend_elements(prop="colors", num=num, fmt=fmt)

# generate the legend entries with color and size
# generation of legends_c_pos must match! sc.legend_elements(..., fmt,...)
fmt_e = fmt.split('{x')[1].split('}', 1)[1]
fmt_s = fmt.split('{x')[0]

legends_c_pos = np.array([i.replace(fmt_e, '').replace(fmt_s, '') for i in legends_c[1]], dtype=float)
legends_c_size = np.sqrt(norm_size(legends_c_pos))

for i, size_i in enumerate(legends_c_pos):
    legends_c[0][i].set_markersize(legends_c_size[i])

plt.legend(*legends_c, title="Size of Cluster", bbox_to_anchor=(1.35, -.018), loc='lower left')
### LEGEND DONE


plt.xlim(0, 960)
plt.ylim(1280, 0)
ax.set_aspect('equal', 'box')

plt.grid()
plt.gca().set_axisbelow(True)  # move grid behind scatter plot
plt.tight_layout()
#plt.savefig('center_of_mass.pdf', bbox_inches='tight')

#### Center of Pixels

In [ ]:
center_of_pix_y = df_cut['center_of_pix_x'].to_numpy()
center_of_pix_x = df_cut['center_of_pix_y'].to_numpy()

### Scatter plot ###
# def. linear norm function for marker size, maps [c.min(), c.max()] -> [s_min, s_max]
# min and max size of markers
s_min=1.
s_max=5e2
norm_size = scipy.interpolate.interp1d([df_cut.n_pixel.min(), df_cut.n_pixel.max()], [s_min, s_max])

# def. cmap, select one
#cmap=strawb.tools.cmap_manipulator('viridis_r', v_min=.03, v_max=1)  # v_min=.03 - cuts the yellow part
cmap=strawb.tools.cmap_manipulator('viridis', v_min=0, v_max=.97)  # v_max=.97 - cuts the yellow part
#cmap=strawb.tools.cmap_manipulator('Blues', v_min=.35, v_max=1)  #  v_min=.3 - cuts the white part


### PLOT
fig, ax = plt.subplots()
sc = plt.scatter(center_of_pix_x, center_of_pix_y, c=df_cut.n_pixel, s=norm_size(df_cut.n_pixel), 
                 cmap=cmap, norm=matplotlib.colors.LogNorm(),
                 alpha=.3)


# circle around the module above the camera
plt.scatter(module_x, module_y, s=200, facecolors='none', edgecolors='red', 
            label='Module above camera', linewidth=1.5) 
# lines for the data cable and steel cable
ax.plot([625, module_x], [300, module_y], 'r-', linewidth=1.5, label='Data cable')
ax.plot([678, module_x], [320, module_y], 'r', linewidth=1.5, linestyle='--', label = 'Steel cable')
ax.legend(loc='lower right', prop={'size':12})

legend1 = ax.legend(loc='lower left', bbox_to_anchor = (1.35, .27))
ax.add_artist(legend1)


# colorbar
cb = plt.colorbar(sc)
cb.set_label("Size of Cluster [Pixel]")


### LEGEND START
# get the legend - maybe there is a better solution
num = matplotlib.ticker.LogLocator(numticks=6).tick_values(df_cut.n_pixel.min(), df_cut.n_pixel.max())

fmt = "{x:.0f} Pixel"
legends_c = sc.legend_elements(prop="colors", num=num, fmt=fmt)

# generate the legend entries with color and size
# generation of legends_c_pos must match! sc.legend_elements(..., fmt,...)
fmt_e = fmt.split('{x')[1].split('}', 1)[1]
fmt_s = fmt.split('{x')[0]

legends_c_pos = np.array([i.replace(fmt_e, '').replace(fmt_s, '') for i in legends_c[1]], dtype=float)
legends_c_size = np.sqrt(norm_size(legends_c_pos))

for i, size_i in enumerate(legends_c_pos):
    legends_c[0][i].set_markersize(legends_c_size[i])

plt.legend(*legends_c, title="Size of Cluster", bbox_to_anchor=(1.35, -.018), loc='lower left')
### LEGEND DONE


plt.xlim(0, 960)
plt.ylim(1280, 0)
ax.set_aspect('equal', 'box')

plt.grid()
plt.gca().set_axisbelow(True)  # move grid behind scatter plot
plt.tight_layout()
#plt.savefig('center_of_mass.pdf', bbox_inches='tight')

#### Center of the Minimum Bounding Boxes

In [ ]:
box_center_y = df_cut['box_center_x'].to_numpy()
box_center_x = df_cut['box_center_y'].to_numpy()

### Scatter plot ###
# def. linear norm function for marker size, maps [c.min(), c.max()] -> [s_min, s_max]
# min and max size of markers
s_min=1.
s_max=5e2
norm_size = scipy.interpolate.interp1d([df_cut.n_pixel.min(), df_cut.n_pixel.max()], [s_min, s_max])

# def. cmap, select one
#cmap=strawb.tools.cmap_manipulator('viridis_r', v_min=.03, v_max=1)  # v_min=.03 - cuts the yellow part
cmap=strawb.tools.cmap_manipulator('viridis', v_min=0, v_max=.97)  # v_max=.97 - cuts the yellow part
#cmap=strawb.tools.cmap_manipulator('Blues', v_min=.35, v_max=1)  #  v_min=.3 - cuts the white part


### PLOT
fig, ax = plt.subplots()
sc = plt.scatter(box_center_x, box_center_y, c=df_cut.n_pixel, s=norm_size(df_cut.n_pixel), 
                 cmap=cmap, norm=matplotlib.colors.LogNorm(),
                 alpha=.3)


# circle around the module above the camera
plt.scatter(module_x, module_y, s=200, facecolors='none', edgecolors='red', 
            label='Module above camera', linewidth=1.5) 
# lines for the data cable and steel cable
ax.plot([625, module_x], [300, module_y], 'r-', linewidth=1.5, label='Data cable')
ax.plot([678, module_x], [320, module_y], 'r', linewidth=1.5, linestyle='--', label = 'Steel cable')
ax.legend(loc='lower right', prop={'size':12})

legend1 = ax.legend(loc='lower left', bbox_to_anchor = (1.35, .27))
ax.add_artist(legend1)


# colorbar
cb = plt.colorbar(sc)
cb.set_label("Size of Cluster [Pixel]")


### LEGEND START
# get the legend - maybe there is a better solution
num = matplotlib.ticker.LogLocator(numticks=6).tick_values(df_cut.n_pixel.min(), df_cut.n_pixel.max())

fmt = "{x:.0f} Pixel"
legends_c = sc.legend_elements(prop="colors", num=num, fmt=fmt)

# generate the legend entries with color and size
# generation of legends_c_pos must match! sc.legend_elements(..., fmt,...)
fmt_e = fmt.split('{x')[1].split('}', 1)[1]
fmt_s = fmt.split('{x')[0]

legends_c_pos = np.array([i.replace(fmt_e, '').replace(fmt_s, '') for i in legends_c[1]], dtype=float)
legends_c_size = np.sqrt(norm_size(legends_c_pos))

for i, size_i in enumerate(legends_c_pos):
    legends_c[0][i].set_markersize(legends_c_size[i])

plt.legend(*legends_c, title="Size of Cluster", bbox_to_anchor=(1.35, -.018), loc='lower left')
### LEGEND DONE


plt.xlim(0, 960)
plt.ylim(1280, 0)
ax.set_aspect('equal', 'box')

plt.grid()
plt.gca().set_axisbelow(True)  # move grid behind scatter plot
plt.tight_layout()
#plt.savefig('center_of_mass.pdf', bbox_inches='tight')

## ----------------------------------------------------------------------------------------------------------

# Time Dependency
Histograms for the total number of pictures taken, the number of pictures with bioluminescent events on them and for the probability that a picture shows an event.

In [ ]:
# define size of the bins

t_tot = pd.to_numeric(image_cluster_db.dataframe.time[~image_cluster_db.dataframe.time.isnull()]) * 1e-9 # convert to seconds

t_tot_u = np.unique(t_tot)
steps = 24*3600  # how many seconds are in one bin
t_start = t_tot_u[0] - t_tot_u[0] % steps # start the bins at midnight
t_end = t_tot_u[-1] - t_tot_u[-1] % steps + steps
bins = np.arange(t_start , t_end, steps)  # number of bins = number of days


# histogram of all pictures

counts_tot, bin_edges_tot = np.histogram(t_tot_u, bins=bins)
counts_tot = np.ma.masked_equal(counts_tot, 0)

date_edges = strawb.tools.asdatetime(bin_edges_tot)

fig, ax = plt.subplots(2, figsize=(6, 7), sharex=True)

plt.sca(ax[0])
plt.xticks(rotation=45)
plt.stairs(counts_tot, date_edges)
#plt.xlabel('Date')
plt.ylabel('Total number of pictures [1/day]')
plt.grid(color='lightgray')
plt.gca().set_axisbelow(True)


# histogram of pictures with animals

t = pd.to_numeric(df_cut.time[mask_para][~image_cluster_db.dataframe.time.isnull()]) * 1e-9 # convert to seconds
t_u = np.unique(t)

counts, bin_edges = np.histogram(t_u, bins=bins)

plt.sca(ax[1])
plt.xticks(rotation=45)
plt.stairs(counts, date_edges)
plt.xlabel('Date')
plt.ylabel("""Pictures with 
bioluminescent events [1/day]""")
plt.grid(color='lightgray')
plt.gca().set_axisbelow(True)
fig.subplots_adjust(hspace=0.1)  # adjust the vertical space between the subplots

#plt.savefig('total_number_of_pictures_and_with_animals.pdf', bbox_inches='tight')

plt.show()


p_animal = (counts/counts_tot)*100  # probability that a picture shows an animal

date = pd.to_datetime(strawb.tools.cal_middle(bin_edges)/(1e-9))

min_time = camera.file_handler.measured_capture_time[:].min() # minimal time needed for exposure + download 
theo_pix_min = (24*3600)/min_time  # maximal number of pictures per day


# probability that a picture shows an animal for each day

fig, ax = plt.subplots()
plt.xticks(rotation=45)
ax.plot(date, p_animal)
plt.xlabel('Date')
plt.ylabel("""Probability that a picture shows 
a bioluminescent event [%]""")

# number of pictures with animals per day if the camera was performing ideally and 
# there was no time needed for the download of the pictures

ax2 = ax.secondary_yaxis('right', 
                         functions = (lambda x: (x/100)* theo_pix_min * (min_time / camera.file_handler.exposure_time[:][0]),
                         lambda x: (x/100)/ (theo_pix_min * (min_time / camera.file_handler.exposure_time[:][0])) ))
ax2.set_ylabel("""Pictures with bioluminescent events
for ideal camera performance [1/day]""")
plt.grid(color='lightgray')
plt.gca().set_axisbelow(True)
#plt.savefig('probability_and_number_of_pictures_ideal.pdf', bbox_inches='tight')
plt.show()

## ----------------------------------------------------------------------------------------------------------